In [3]:
# Required packages to run the script
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

import seaborn as sn
from scipy.stats import pearsonr

In [4]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 's:\\131-DPE\\131.1-Temporaire\\131.1.1-Espace commun\\zone transfert temporaire\\ITESE Grenoble\\Stage Justine\\Code git\\wavelet_decomposition')
from wavelet_decomposition import reconstruct

In [5]:
path_input_data = '../input_time_series/'
dpd = 24 # data per day in the time serie
dpy = 365 # data per year :  cut the leap years to 365 years

ndpd = 64 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

# Chargement des caractéristiques des signaux

In [7]:
import seaborn as sn
from scipy.stats import pearsonr
list_lags = pd.read_csv("results/lags_world_diff_true.csv", index_col=0)

for country_name in list_lags.index[:] :
    try :
        result_optim = pd.read_pickle(f'results/{country_name}/optimization_results.pickle')
        list_lags.loc[country_name,'share_wind'] = result_optim['share_wind']
        list_lags.loc[country_name,'share_pv'] = result_optim['share_pv']
        list_lags.loc[country_name,'share_dispatch'] = result_optim['share_dispatchable']
        list_lags.loc[country_name, 'cf_pv']=np.min(result_optim['cf_pv'])
        list_lags.loc[country_name, 'cf_wind'] = np.min(result_optim['cf_wind'])
        list_lags.loc[country_name, 'corr_load_wind']= pearsonr(result_optim['consumption'], result_optim['optimized_wind'])[0]
        list_lags.loc[country_name, 'corr_load_pv']= pearsonr(result_optim['consumption'], result_optim['optimized_pv'])[0]
        # Les résultats des décompositions pour chaque signal de conso et de production ont été calculés et sont stcokés dans
        # un fichier .pickle par pays.
        data_wavelets = pd.read_pickle( f'results_decomposition_coefficient/Plexos/wavelets_reconstruction/wavelets_{country_name}_corrected.pickle')
        # LOAD
        list_lags.loc[country_name, 'beta_year_load'] = np.max(data_wavelets['load_year'])
        list_lags.loc[country_name, 'max_beta_week_load'] = np.max(data_wavelets['load_week'])
        list_lags.loc[country_name, 'max_beta_day_load'] = np.max(data_wavelets['load_day'])
        pv_week = data_wavelets['load_week']
        pv_week = pv_week[pv_week>0]
        list_lags.loc[country_name, 'mean_beta_week_load'] = np.mean(data_wavelets['load_week'][data_wavelets['load_week']>0])
        list_lags.loc[country_name, 'mean_beta_day_load'] = np.mean(data_wavelets['load_day'][data_wavelets['load_day']>0])

        # PV
        list_lags.loc[country_name, 'beta_year_pv'] = np.max(data_wavelets['pv_year'])
        list_lags.loc[country_name, 'max_beta_week_pv'] = np.max(data_wavelets['pv_week'])
        list_lags.loc[country_name, 'max_beta_day_pv'] = np.max(data_wavelets['pv_day'])
        list_lags.loc[country_name, 'mean_beta_week_pv'] = np.mean(data_wavelets['pv_week'][data_wavelets['pv_week']>0])
        list_lags.loc[country_name, 'mean_beta_day_pv'] = np.mean(data_wavelets['pv_day'][data_wavelets['pv_day']>0])

        # WIND
        list_lags.loc[country_name, 'beta_year_wind'] = np.max(data_wavelets['wind_year'])
        list_lags.loc[country_name, 'max_beta_week_wind'] = np.max(data_wavelets['wind_week'])
        list_lags.loc[country_name, 'max_beta_day_wind'] = np.max(data_wavelets['wind_day'])
        list_lags.loc[country_name, 'mean_beta_week_wind'] = np.mean(data_wavelets['wind_week'][data_wavelets['wind_week']>0])
        list_lags.loc[country_name, 'mean_beta_day_wind'] = np.mean(data_wavelets['wind_day'][data_wavelets['wind_day']>0])

        # SCALAR PRODUCTS
        list_lags.loc[country_name, 'scalar_year_pv_load'] = np.max(data_wavelets['scalar_year_pv_load'])
        list_lags.loc[country_name, 'scalar_year_wind_load'] = np.max(data_wavelets['scalar_year_wind_load'])
        list_lags.loc[country_name, 'scalar_day_pv_load'] = np.max(data_wavelets['scalar_day_pv_load'])
        list_lags.loc[country_name, 'scalar_day_wind_load'] = np.mean(data_wavelets['scalar_day_wind_load'])
        list_lags.loc[country_name, 'scalar_week_pv_load'] = np.mean(data_wavelets['scalar_week_pv_load'])
        list_lags.loc[country_name, 'scalar_week_wind_load'] = np.mean(data_wavelets['scalar_week_pv_load'])

        # WEIGHT TS
        list_lags.loc[country_name, 'weight_pv'] = result_optim['pv_capacity']/result_optim['mean_consumption']
        list_lags.loc[country_name, 'weight_wind'] = result_optim['wind_capacity']/result_optim['mean_consumption']

    except Exception as e:
        print(f"Error processing {country_name}: {e}")

list_lags.to_csv("results/predictors_regression_country_cf_corrected.csv")

Error processing Antarctica: 'share_wind'
Error processing Dominican Republic: 'share_wind'
Error processing Rwanda: 'share_wind'
Error processing Swaziland: 'share_wind'
Error processing Syrian Arab Republic: 'share_wind'
Error processing Turkmenistan: 'share_wind'
Error processing United Arab Emirates: 'share_wind'
Error processing Western Sahara: 'share_wind'


In [8]:
list_lags_states = pd.read_csv("results/lags_world_diff_states_3.csv").drop_duplicates()
regions_match = pd.read_excel('match_plexos_iso_codes.xlsx', index_col=1)

list_lags_states = list_lags_states.set_index(['Country', 'State'])
list_lags_states = list_lags_states.sort_index()
for country, state_name in list_lags_states.index :
    regions_match_country = regions_match[regions_match['Country']==country]
    # print(country, state_name)

    region_name = regions_match_country.loc[state_name, 'plexos']
    # print(region_name)
    try :
        result_optim = pd.read_pickle(f'results/{country}/optimization_results_{state_name}.pickle')
        # print(list_lags_states.loc[(country, state_name),'share_wind'])
        list_lags_states.loc[(country, state_name),'share_wind'] = result_optim['share_wind']
        # print(list_lags_states.loc[(country, state_name),'share_wind'])
        list_lags_states.loc[(country, state_name),'share_pv'] = result_optim['share_pv']
        list_lags_states.loc[(country, state_name),'share_dispatch'] = result_optim['share_dispatchable']
        list_lags_states.loc[(country, state_name), 'cf_pv']=np.min(result_optim['cf_pv'])
        list_lags_states.loc[(country, state_name), 'cf_wind'] = np.min(result_optim['cf_wind'])
    
        list_lags_states.loc[(country, state_name), 'corr_load_wind']= pearsonr(result_optim['consumption'], result_optim['optimized_wind'])[0]
        list_lags_states.loc[(country, state_name), 'corr_load_pv']= pearsonr(result_optim['consumption'], result_optim['optimized_pv'])[0]
        # Les résultats des décompositions pour chaque signal de conso et de production ont été calculés et sont stcokés dans
        # un fichier .pickle par pays.
        # print('Now wavelets...')
        data_wavelets = pd.read_pickle( f'results_decomposition_coefficient/Plexos/wavelets_reconstruction/wavelets_{country}_{state_name}_corrected.pickle')
       # LOAD
        list_lags_states.loc[(country, state_name), 'beta_year_load'] = np.max(data_wavelets['load_year'])
        list_lags_states.loc[(country, state_name), 'max_beta_week_load'] = np.max(data_wavelets['load_week'])
        list_lags_states.loc[(country, state_name), 'max_beta_day_load'] = np.max(data_wavelets['load_day'])
    
        list_lags_states.loc[(country, state_name), 'mean_beta_week_load'] = np.mean(data_wavelets['load_week'][data_wavelets['load_week']>0])
        list_lags_states.loc[(country, state_name), 'mean_beta_day_load'] = np.mean(data_wavelets['load_day'][data_wavelets['load_day']>0])

        # PV
        list_lags_states.loc[(country, state_name), 'beta_year_pv'] = np.max(data_wavelets['pv_year'])
        list_lags_states.loc[(country, state_name), 'max_beta_week_pv'] = np.max(data_wavelets['pv_week'])
        list_lags_states.loc[(country, state_name), 'max_beta_day_pv'] = np.max(data_wavelets['pv_day'])
        list_lags_states.loc[(country, state_name), 'mean_beta_week_pv'] = np.mean(data_wavelets['pv_week'][data_wavelets['pv_week']>0])
        list_lags_states.loc[(country, state_name), 'mean_beta_day_pv'] = np.mean(data_wavelets['pv_day'][data_wavelets['pv_day']>0])

        # WIND
        list_lags_states.loc[(country, state_name), 'beta_year_wind'] = np.max(data_wavelets['wind_year'])
        list_lags_states.loc[(country, state_name), 'max_beta_week_wind'] = np.max(data_wavelets['wind_week'])
        list_lags_states.loc[(country, state_name), 'max_beta_day_wind'] = np.max(data_wavelets['wind_day'])
        list_lags_states.loc[(country, state_name), 'mean_beta_week_wind'] = np.mean(data_wavelets['wind_week'][data_wavelets['wind_week']>0])
        list_lags_states.loc[(country, state_name), 'mean_beta_day_wind'] = np.mean(data_wavelets['wind_day'][data_wavelets['wind_day']>0])
        # SCALAR PRODUCTS
        list_lags_states.loc[(country, state_name), 'scalar_year_pv_load'] = np.max(data_wavelets['scalar_year_pv_load'])
        list_lags_states.loc[(country, state_name), 'scalar_year_wind_load'] = np.max(data_wavelets['scalar_year_wind_load'])
        list_lags_states.loc[(country, state_name), 'scalar_day_pv_load'] = np.max(data_wavelets['scalar_day_pv_load'])
        list_lags_states.loc[(country, state_name), 'scalar_day_wind_load'] = np.mean(data_wavelets['scalar_day_wind_load'])
        list_lags_states.loc[(country, state_name), 'scalar_week_pv_load'] = np.mean(data_wavelets['scalar_week_pv_load'])
        list_lags_states.loc[(country, state_name), 'scalar_week_wind_load'] = np.mean(data_wavelets['scalar_week_wind_load'])

        # WEIGHT TS
        list_lags_states.loc[(country, state_name), 'weight_pv'] = result_optim['pv_capacity']/result_optim['mean_consumption']
        list_lags_states.loc[(country, state_name), 'weight_wind'] = result_optim['wind_capacity']/result_optim['mean_consumption']

        # betas = pd.read_excel(f'results_decomposition_coefficient/Plexos/{country_name}/results_betas_stacked.xlsx', header=None)
        # list_lags.loc[country_name, 'beta_year'] = np.max(betas.iloc[:,2])
        # list_lags.loc[country_name, 'max_beta_week'] = np.max(betas.iloc[:,1])
        # list_lags.loc[country_name, 'max_beta_day'] = np.max(betas.iloc[:,0])
        # list_lags.loc[country_name, 'beta_year'] = np.max(betas.iloc[:,2])
        # list_lags.loc[country_name, 'mean_beta_week'] = np.mean(betas.iloc[:,1])
        # list_lags.loc[country_name, 'mean_beta_day'] = np.mean(betas.iloc[:,0])

    except Exception as e:
        print(f"Error processing {country} and {state_name}: {e}")

list_lags_states.to_csv("results/predictors_regression_states_cf_corrected.csv")

Error processing Canada and NL: 'share_wind'


# Debug pays précédents

In [336]:
result_optim = pd.read_pickle(f'results/United Arab Emirates/optimization_results.pickle')
betas = pd.read_excel(f'results_decomposition_coefficient/Plexos/United Arab Emirates/results_betas_stacked.xlsx', header=None)
result_optim.keys()

dict_keys(['optimized_pv', 'optimized_wind', 'optimized_dispatchable', 'optimized_stock', 'optimized_charge', 'optimized_discharge', 'optimized_p_curt', 'consumption', 'mean_consumption', 'pv_capacity', 'wind_capacity', 'dispatchable_capacity', 'E_wind', 'E_pv', 'E_dispatch', 'E_curt', 'E_loss', 'E_stock', 'E_destock', 'iso_alpha'])

On remarque que la variable 'share_pv' a un problème de facteur 10... On met à jour tous les fichiers .pickle dans la cellule suivante. **Il ne faut pas faire tourner cette cellule à nouveau.**

In [337]:
# import pickle
# for country_name in list_lags.index[:] :
#     try:
#         result_optim = pd.read_pickle(f'results/{country_name}/optimization_results.pickle')
#         if result_optim['share_pv'] >=100:
#             result_optim['share_pv']/=10
#         # result_optim['share_pv']*=10
#         with open(f'results/{country_name}/optimization_results.pickle', 'wb') as pickle_file:
#             pickle.dump(result_optim, pickle_file)
#     except Exception as e:
#         print(f"Error processing {country_name}: {e}")

In [339]:
result_optim = pd.read_pickle(f'results/Australia/optimization_results.pickle')
betas = pd.read_excel(f'results_decomposition_coefficient/Plexos/Australia/results_betas_stacked.xlsx', header=None)
result_optim

{'optimized_pv': [1.163315525799841,
  1.4759245437460538,
  1.6706645877125472,
  1.7552227646979983,
  1.7321614437019661,
  1.6066053627235692,
  1.3401189867694205,
  0.99163680282938,
  0.5867825008990388,
  0.2024271509651706,
  0.012811844997795608,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.04356027299250507,
  0.29979717294841723,
  0.6072814528955117,
  0.9762625888320253,
  1.3016834517760336,
  1.5143600787394407,
  1.5861064107270961,
  1.6399161597178378,
  1.5912311487262145,
  1.388803997761044,
  1.0608207658174762,
  0.6303427738915439,
  0.19474004396649322,
  0.012811844997795608,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.03843553499338682,
  0.2972348039488581,
  0.7866472828646502,
  1.222250012789701,
  1.4707998057469356,
  1.6194172077213649,
  1.6809140637107838,
  1.6527280047156334,
  1.5194848167385588,
  1.3119329277742702,
  1.012135754825853,
  0.6175309288937483,
  0.2024271509651706,
  0.01